In [1]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import matplotlib.pyplot as plt

import pymorphy2
import re
from collections import defaultdict

from tqdm import tqdm, tqdm_notebook
import bisect
from collections import namedtuple,defaultdict
import numpy as np

import requests
import sqlite3


import time
import pickle
#MS SQL 
import pyodbc

In [48]:
#MS SQL
server = 'NUTS' 
#database = 'ThesisMK' 
database = 'MK_news_1000'
username = '' 
password = '' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cur_sql = cnxn.cursor()

In [65]:

stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот','ещё', 'свой' , 'быть','й', 'как', 'в', '—', 'к', 'на','свой', 'который', 'е','кстати', 'также'])

def parse_text(text):
    #print(type(text))
    #words = (word for word in re.split('\W+', text) if (len(word) >0 )&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    words = (word for word in re.split('\W+', text) if (len(word) >0 )&(re.match(r'(\D+$)',word)is not None)&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)


class InvertedIndex:
    
    def __init__(self):
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, text, doc_id):
        self.texts[doc_id] = text
        words = parse_text(text)
        
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
            
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
        
            postings.insert(i, entry)
            
    def get_postings(self, word):
        return self.dict[word]
    

        
"""Подсчет векторного произведения"""        
def mult_single(x,y):
    #conditions x>0 y>o
    return x*y

def mult_vect(x,y):
    mult = np.vectorize(mult_single)
    return np.sum(mult(x,y))

"""Dkl"""

def dkl_single(x,y):
    if x>0 and y>0:
     
        return x*math.log2(x/y)
    elif x==0: return 0.0
    else: return math.inf

def dkl(x,y):
    sum_dkl = np.vectorize(dkl_single)
    
    return np.sum(sum_dkl(x,y))

def div_matr(D, n, m,py_x,py_t) :
    for i in range(n):
        for j in range(m):
            Div[i,j] = dkl(py_x[:,i],py_t.transpose()[:,j]) 
            
    return Div
    
"""Djs"""
def djs(x,y):
    return 0.5 * dkl(x, (x+y)/2) + 0.5* dkl(y, (x+y)/2)

def djs_vect(Djs,m,pt_x,pt_x_new ):
    for i in range(m):
        Djs[i]= djs(pt_x[:,i], pt_x_new[:,i])
        
        #Djs[i]= djs(pt_x[i,:], pt_x_new[i,:])
    return Djs

"""P(t)"""
def f_pt(pt,m,px,pt_x):
    for i in range(m):
        pt[i] = mult_vect(px, pt_x[i,:])
    return pt

"""P(y|t)"""
def f_py_t(py_t, m,n,pt_x,pxy,pt):
    for i in range(m):
        for j in range(n):
            py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]        
    return py_t
"""Выводит список из  n элементов, соответсвующих  """
def main_words(n, r):
    l = []

    for i in range(n):
        l.append(r.argmax())
        r[r.argmax()] = 0
    return l


        
#show all numbers of docs from  cl,   
def show_text_docs(cl_num):
    show_main_words(cl_num)
    print("\n")
    docs = doc_clast[cl_num] # список документов кластера №cl_num
    for num in docs:
        print("№ документа: " + str(num) + " ТЕКСТ "+ str(x[num]))
    
    
    
def foo_z(pt_x):
    z = pt_x.sum(axis=0)
    return z


#w_d список документов со словом w
#d_cl список документов из кластера t 

#w_d,  d_cl
def f_N11( w_d,d_cl,n):
    k= 0
    for d in range(n):
        if (d in d_cl )& (d in w_d):
            k+=1
    return k
#not w_d, w_cl
def f_N01(w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d in w_cl )& (d not in w_d):
            k+=1
    return k
#not w_d, not w_cl
def f_N00( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d not in w_d):
            k+=1
    return k

#w_d,   not w_cl
def f_N10( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d in w_d):
            k+=1
    return k

def mutual_information(t):
    MI = []
    for word in tqdm(word_dict.items()):
        w  = word[1]
        
        w_d=[]
        for doc_id,pos in index.dict.get(w):
            w_d.append(doc_id)
        N = len(x)
        N11 = f_N11(w_d,doc_clast.get(t),len(x))
        N1 = len(w_d)
        N10 = f_N10(w_d,doc_clast.get(t),len(x))
        N0 = N - N1
        #print(N, N1, N0)
        N01 = f_N01(w_d,doc_clast.get(t),len(x))
        N00 = f_N00(w_d,doc_clast.get(t),len(x))
        if N10==0: N10=0.0001
        if N01==0: N01=0.0001
        if N11==0: N11=0.0001
        if N00==0: N00=0.0001
   
        #print(N11, N01, N10, N00)
        I = N11/N * math.log2(N*N11/(N1*N1)) + N01/N*math.log2(N * N01/(N0*N1)) +N10/N*math.log2(N * N10/(N0*N1))+N00/N*math.log2(N * N00/(N0*N0))
        MI.append(I)
    return MI
    
def main_words(n, r):
    l = []
    
    for i in range(n): 
        maxi = r.argmax()
        l.append((maxi,r[maxi]))
        r[maxi] = 0
    return l


def replaceZeroes(data):
    #print(data)
    data[data == 0] = 0.0001    
    return data

def replaceZeroeScalar(data):
    if data == 0:
        data = 0.0001   
    
    #print(data)
    return data

def IXT(px,pt_x,pt,m,n):
    IXT = 0
    for t in range(m):
        #for x in range(n): #n len(x)
            #print(replaceZeroes([1,3,0,7]))
            a = px*pt_x[t,:]*np.log2(replaceZeroes(pt_x[t,:]/replaceZeroeScalar(pt[t])))
            #print(a.sum())
            IXT+=a.sum()
    
    print("IXT = " + str(IXT))
    return IXT

def ITY_py_t(py_t,pxy,pt, m, k):
    ITY = 0    
    py = pxy.sum(axis=0)
    for t in range(m):
        #for y in range(k):
            a = pt[t] * py_t[t,:]*np.log2(replaceZeroes(py_t[t,:]/replaceZeroes(py)))
            ITY+=a.sum()   
            #print(ITY)
    
    print("ITY = " + str(ITY))
    return ITY

def plot_L(Func_L):
    t = np.arange(0, len(Func_L), 1)
    L = np.array(Func_L)
    plt.plot(t, L)

    plt.xlabel('iteration (k)')
    plt.ylabel('L[p(t|x)]')
    plt.title('I(X;T)-beta*I(T;Y)')
    plt.grid(True)
    plt.savefig("test.png")
    plt.show()

def f_sort(n, r):
    l = []
    for i in range(n):
        maxi = r.argmax()
        l.append(maxi)
        r[maxi] = 0
    return l

def sent_weight(l,number_main_words):  #возвращает вес предложения
    weights=[]
    for word in l:
        weights.append(mi_dict[new_word_dict[word]][0])
    weights = np.array(weights)
    weight = np.sum(weights[:number_main_words])
    return weight


def Q_foo(document):
    return 2/3*len(parse_text(document)/sent_tokenize(document))
    
    

        
    

In [59]:
def summary_1(choice_document):
    cmd = "Select doc_text from Documents where id=?"
    cur_sql.execute(cmd,choice_document)
    data = cur_sql.fetchone()[0]
    cmd = "Select claster_id from Doc_to_claster where doc_id=?"
    cur_sql.execute(cmd,choice_document)
    cluster = cur_sql.fetchone()[0]
    V = math.ceil(sent_tokenize(choice_document))
    #MI_(num)
    cmd = """DECLARE @table_name sysname SELECT @table_name = 'MI_'+? 
    EXEC( 'select mi from ' + @table_name )"""
    cur_sql.execute(cmd,cluster)
    MI = cur_sql.fetchall()

    #словарь слово ключ : индекс слова, значение : вес
    mi_dict={}
    for i, mi in enumerate(MI):
        mi_dict[i] = mi

    # переопределение словаря, теперь ключ: слово, значение :индекс
    new_word_dict = {}
    cmd = "select id, name from Words"
    cur_sql.execute(cmd)
    for i, word in tqdm(cur_sql.fetchall()):
        new_word_dict[word] = i 
        
        
    sentences = sent_tokenize(data)
    print("Number of sent = "+str(len(sentences)))
    sentence_weight=[]
    for n,sent in enumerate(sentences):
        parsed_text = parse_text(sent)
        if len(parsed_text) > Q_foo(choice_document):      
            sentence_weight.append(sent_weight(parsed_text,Q_foo(choice_document)))
        else: 
            weight = 0
            for word in parsed_text:
                weight+=(mi_dict[new_word_dict[word]])
            sentence_weight.append(weight)
        
            
    sentence_weight = np.array(sentence_weight)
    ss = f_sort(V,sentence_weight.copy())
    ss = sorted(ss)
    #for i in ss:
    #    print(i,sentence_weight[i], sentences[i], len(word_tokenize(sentences[i])))   
    return ss
        
        
def summary_0(choice_document ):
    ref=[]
    cmd = "Select doc_text from Documents where id=?"
    cur_sql.execute(cmd,choice_document)
    data = cur_sql.fetchone()[0]
    cmd = "select * from Doc_to_claster_probability where doc_id = ?"
    cur_sql.execute(cmd,num)
    V = math.ceil(sent_tokenize(choice_document))
    distr=[]
    for i in cur_sql.fetchone()[1:]:
        distr.append(i)
    distr = np.array(distr)
    for cluster, prob in enumerate(distr):
        cmd = """DECLARE @table_name sysname SELECT @table_name = 'MI_'+? 
        EXEC( 'select mi from ' + @table_name ) """
        cur_sql.execute(cmd,cluster)
        MI = cur_sql.fetchall()

        #словарь слово ключ : индекс слова, значение : вес
        mi_dict={}
        for i, mi in enumerate(MI):
            mi_dict[i] = mi

        # переопределение словаря, теперь ключ: слово, значение :индекс
        new_word_dict = {}
        cmd = "select id, name from Words"
        cur_sql.execute(cmd)
        for i, word in tqdm(cur_sql.fetchall()):
            new_word_dict[word] = i    
        
        sentences = sent_tokenize(data)
        print("Number of sent = "+str(len(sentences)))
        sentence_weight=[]
        for n,sent in enumerate(sentences):
            parsed_text = parse_text(sent)
            if len(parsed_text) > Q_foo(choice_document):      
                sentence_weight.append(sent_weight(parsed_text,N))
            else: 
                weight = 0
                for word in parsed_text:
                    weight+=(mi_dict[new_word_dict[word]])
                sentence_weight.append(weight)
        
            
        sentence_weight = np.array(sentence_weight)
        ss = f_sort(math.ceil(V*distr),sentence_weight.copy())
        ss = sorted(ss)
        ref = list(set(ref + ss))
        #for i in ss:
        #    print(i,sentence_weight[i], sentences[i], len(word_tokenize(sentences[i])))   
    return ref
        
        
    

In [29]:
def information_bottleneck(beta, m):
    start_time = time.clock()

 
        
    Func_L = []
        
    #словарь слов: ключ - номер индекса , значение -словао
    word_dict = {}
    for i, word in enumerate(index.dict.keys()):
        word_dict[i] = word
    docs_lens=[]
    """#общее число слов в коллекции"""
    summa = 0
    for doc in tqdm(x):
        length = len(parse_text(doc))
        docs_lens.append(length)
        summa+= length
        print(summa)

    docs_lens = np.array(docs_lens)
    px=docs_lens/summa
    print(px.sum())
    """P(x,y)"""    

    pxy = np.zeros((len(x), len(index.dict.keys())))
    for i, word in enumerate(index.dict.keys()):
        a = index.dict.get(word)
        for doc_id , postings in a:
            pxy[doc_id,i] = len(postings)/summa
        
    print("pxy + ")
    print(pxy.sum())  
    """
    print(pxy)"""     
        
    """INITIALIZaTION""" 

    """P(t|x)"""

    pt_x = np.random.rand(m, len(x))

    z = pt_x.sum(axis=0)
    pt_x = pt_x/z
    #print(pt_x)
    


    """P(t)"""
    pt = np.zeros(m)
    for i in range(m):
        pt[i] = mult_vect(px,pt_x[i,:])
    
    """P(y|t)"""
    py_t = np.zeros((m,len(index.dict.keys())) )

    for i in range(m):
        for j in range(len(index.dict.keys())):
            py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
        
    print("py_t +")
    print(py_t.sum(axis=1))
    """P(y|x)"""
    py_x = np.zeros((len(index.dict.keys()),len(x)))


    word_index=[]
    for i, word in tqdm(enumerate(index.dict.keys())):
        a = index.dict.get(word)
        word_index.append((i,word))
        for doc_id , postings in a:
            py_x[i,doc_id] = len(postings)/docs_lens[doc_id]        
    print("py_x + ")
    print(py_x.sum(axis=0))
      
    """LOOP"""
    Div = np.zeros((len(x), m)) #init div   
    pt_x_new = pt * np.exp(-beta * div_matr(Div, len(x),m,py_x,py_t)) 

    pt_x_new = pt_x_new.transpose()
    z = foo_z(pt_x_new)
    pt_x_new = pt_x_new/z


    #init Djs
    Djs = np.zeros(len(x))
    Djs = djs_vect(Djs,len(x),pt_x,pt_x_new )

    k = 0

                                                                  
    L = IXT(px,pt_x_new,pt,m, len(x)) - beta* ITY_py_t(py_t,pxy,pt, m, len(index.dict.keys()))

    print(L)                                                                      
    Func_L.append(L)
    print("Djs")
    print(Djs.max())                                                                      
    print (time.clock() - start_time, "seconds")

    while Djs.max()>=e: 
        start_time = time.clock()
        print(k, "iteration")
        #переопрделяем 
        pt_x = pt_x_new 
   
        pt = f_pt(pt,m,px,pt_x)
    
        #P(y|t)
        py_t = f_py_t(py_t, m,len(index.dict.keys()),pt_x,pxy,pt) 
  
        print("py_t +")
        print(py_t.sum(axis=1))             
        #P(t|x) new
        pt_x_new = pt * np.exp(-beta * div_matr(Div, len(x),m,py_x,py_t)) 
        """print('pt_new')
        print(pt_x_new)"""
        pt_x_new = pt_x_new.transpose()
        z = foo_z(pt_x_new)
        pt_x_new = pt_x_new/z
        print("pt_x_new")
       
        Djs = djs_vect(Djs,len(x),pt_x,pt_x_new )
        print("DJS")
        print(Djs.max())
        k+=1
        print("pt")
        print(pt)
        L = IXT(px,pt_x_new,pt,m, len(x)) - beta* ITY_py_t(py_t,pxy,pt, m, len(index.dict.keys()))
    
        print("L = "+str(L))                                                                      
        Func_L.append(L)                                                                  
        print (time.clock() - start_time, "seconds")
    
    # вектор с номерами кластеров, к которому документ предложит( вероятность больше всего)
    vec = np.argmax(pt_x_new, axis=0)

    #словарь: ключ - номер кластера, значение - номер документа
    doc_clast = defaultdict(list)
    for i,value in enumerate(vec):
            doc_clast[value].append(i)
    plot(Func_L)


In [ ]:
#beta
beta = 2.4

#число кластеров
m = 5
#covergence parameter 
e = 5e-04

cmd = "select doc_text from Documents "
cursor.execute(cmd)
d=[]
for t in cursor.fetchall()[:30]:      
        d.append(t[0])
x = np.array(d)       
        
analyzer = pymorphy2.MorphAnalyzer()  
DocEntry = namedtuple('DocEntry',['doc_id','positions'])
"""#Создание инвертированного индекса"""   
index = InvertedIndex()

for i, line in tqdm(enumerate(x)):
        doc_id = i
        text = line
        index.add_document(text,doc_id)
        
information_bottleneck(beta, m)    

# заполнение таблицы слов
words = word_dict.items()
for w in tqdm(words):
    cmd = """insert into Words (name) values (?) """
    cursor.execute(cmd, w[1])   
    cursor.commit() 
    
# заполнение таблицы кластеров
for i in range(m):
    cmd = """insert into Clasters (name) values (?) """
    cursor.execute(cmd, ('name'+str(i)))
    cursor.commit()
# заполнение таблицы кластер-документ    
for i in range(m):
    l = doc_clast[i]
    for number in l:
        cmd = """insert into  Doc_to_claster (doc_id,claster_id) values (?,?)"""
        cursor.execute(cmd, (number, i))
        cursor.commit()

# распределение по кластерам
for i,row in enumerate( pt_x.transpose()):
    cmd = """insert into Doc_to_claster_probability (doc_id,p0,p1,p2,p3,p4,p5,p6) values (?,?,?,?,?,?,?,?) """
    cursor.execute(cmd, (i,row[0],row[1],row[2],row[3],row[4],row[5],row[6]))
    cursor.commit()

    

for cluster in m:
    MI = mutual_information(cluster)
    for i,mi in tqdm(enumerate(MI)):
        cmd = """DECLARE @table_name sysname SELECT @table_name = 'MI_'+?
        EXEC( 'insert into ' + @table_name (word_id, mi) values (?,?) )"""
        cursor.execute(cmd, (str(cluster), i,mi))  
        cursor.commit()
  


In [ ]:
choice_document = 5
type_of_summary = 1 #1 если 1 кластер, 0 - если несколько кластеров
if type_of_summary == 1:
    summary_1(choice_document )
else : 
    summary_0(choice_document )